In [2]:
#load "Paket.fsx"

In [22]:
Paket.Dependencies.Install """
frameworks: net45
source https://nuget.org/api/v2
nuget FSLab
"""

Paket.LoadingScripts.ScriptGeneration.generateScriptsForRootFolder 
   Paket.LoadingScripts.ScriptGeneration.FSharp
   (Paket.FrameworkIdentifier.DotNetFramework Paket.FrameworkVersion.V4_5)
   (System.IO.DirectoryInfo __SOURCE_DIRECTORY__)

<null>

In [23]:
#load "paket-files/include-scripts/net45/include.main.group.fsx"

In [24]:
#r "FSharp.Markdown.dll"
#r "FSharp.Literate.dll"
#r "Deedle.dll"

open System.IO
open Deedle
open Deedle.Internal
open FSharp.Literate
open FSharp.Markdown

// --------------------------------------------------------------------------------------
// Implements Markdown formatters for common FsLab things - including Deedle series
// and frames, F# Charting charts and System.Image values
// --------------------------------------------------------------------------------------
// How many columns and rows from frame should be rendered
let startColumnCount = 3
let endColumnCount = 3
let startRowCount = 8
let endRowCount = 4
// How many items from a series should be rendered
let startItemCount = 5
let endItemCount = 3

// --------------------------------------------------------------------------------------
// Helper functions etc.
// --------------------------------------------------------------------------------------
/// Extract values from any series using reflection
let (|SeriesValues|_|) (value : obj) = 
    let iser = value.GetType().GetInterface("ISeries`1")
    if iser <> null then 
        let keys = 
            value.GetType().GetProperty("Keys").GetValue(value) :?> System.Collections.IEnumerable
        let vector = value.GetType().GetProperty("Vector").GetValue(value) :?> IVector
        Some(Seq.zip (Seq.cast<obj> keys) vector.ObjectSequence)
    else None

/// Format value as a single-literal paragraph
let formatValue def = 
    function 
    | Some v -> [ Paragraph [ Literal(v.ToString()) ] ]
    | _ -> [ Paragraph [ Literal def ] ]

/// Format body of a single table cell
let td v = [ Paragraph [ Literal v ] ]

/// Use 'f' to transform all values, then call 'g' with Some for 
/// values to show and None for "..." in the middle
let mapSteps (startCount, endCount) f g input = 
    input
    |> Seq.map f
    |> Seq.startAndEnd startCount endCount
    |> Seq.map (function 
           | Choice1Of3 v | Choice3Of3 v -> g (Some v)
           | _ -> g None)
    |> List.ofSeq

// Tuples with the counts, for easy use later on
let fcols = startColumnCount, endColumnCount
let frows = startRowCount, endRowCount
let sitms = startItemCount, endItemCount

let getHtml (value : obj) = 
    match value with
    | SeriesValues s -> 
        // Pretty print series!
        let heads = 
            s |> mapSteps sitms fst (function 
                     | Some k -> td (k.ToString())
                     | _ -> td " ... ")
        
        let row = 
            s |> mapSteps sitms snd (function 
                     | Some v -> formatValue "N/A" (OptionalValue.asOption v)
                     | _ -> td " ... ")
        
        let aligns = s |> mapSteps sitms id (fun _ -> AlignDefault)
        [ InlineBlock "<div class=\"deedleseries\">"
          TableBlock(Some((td "Keys") :: heads), AlignDefault :: aligns, [ (td "Values") :: row ])
          InlineBlock "</div>" ]
        |> Some
    | :? IFrame as f -> 
        { // Pretty print frame!
          new IFrameOperation<_> with
              member x.Invoke(f) = 
                  let heads = 
                      f.ColumnKeys |> mapSteps fcols id (function 
                                          | Some k -> td (k.ToString())
                                          | _ -> td " ... ")
                  
                  let aligns = f.ColumnKeys |> mapSteps fcols id (fun _ -> AlignDefault)
                  
                  let rows = 
                      f.Rows
                      |> Series.observationsAll
                      |> mapSteps frows id (fun item -> 
                             let def, k, data = 
                                 match item with
                                 | Some(k, Some d) -> 
                                     "N/A", k.ToString(), Series.observationsAll d |> Seq.map snd
                                 | Some(k, _) -> 
                                     "N/A", k.ToString(), f.ColumnKeys |> Seq.map (fun _ -> None)
                                 | None -> " ... ", " ... ", f.ColumnKeys |> Seq.map (fun _ -> None)
                             
                             let row = 
                                 data |> mapSteps fcols id (function 
                                             | Some v -> formatValue def v
                                             | _ -> td " ... ")
                             
                             (td k) :: row)
                  Some [ InlineBlock "<div class=\"deedleframe\">"
                         TableBlock(Some([] :: heads), AlignDefault :: aligns, rows)
                         InlineBlock "</div>" ] }
        |> f.Apply
    | _ -> None
    
    |> function
       | Some pars -> 
            let doc = MarkdownDocument(pars, null)
            Markdown.WriteHtml(doc)
       | None -> ""

input.fsx(1,1): error FS0082: Could not resolve this reference. Could not locate the assembly "FSharp.Markdown.dll". Check to make sure the assembly exists on disk. If this reference is required by your code, you may get compilation errors. (Code=MSB3245)


input.fsx(1,1): error FS0084: Assembly reference 'FSharp.Markdown.dll' was not found or is invalid

In [14]:
open Deedle
open FSharp.Charting
open System

In [16]:
let dates =
    [ DateTime(2013, 1, 1);
      DateTime(2013, 1, 4);
      DateTime(2013, 1, 8) ]
let values = 
    [ 10.0; 20.0; 30.0 ]

Series(dates, values)

series [ 01/01/2013 00:00:00 => 10; 01/04/2013 00:00:00 => 20; 01/08/2013 00:00:00 => 30]